# Amazon SageMaker Clarify Model Explainability Monitor for Batch Transform - JSON Format

---

This notebook's CI test result for us-west-2 is as follows. CI test results in other regions can be found at the end of the notebook.

![This us-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-2/sagemaker_model_monitor|fairness_and_explainability_json|SageMaker-Monitoring-Feature-Attribution-Drift-for-Batch-Transform.ipynb)

---

## Runtime

This notebook takes approximately 60 minutes to run.

## Contents

* [Introduction](#Introduction)
* [General Setup](#General-Setup)
    * [Imports](#Imports)
    * [Handful of configuration](#Handful-of-configuration)
    * [Data files](#Data-files)
    * [SageMaker model](#SageMaker-model)
* [Batch Transform Job](#Batch-Transform-Job)
    * [Captured data](#Captured-data)
    * [Transform input](#Transform-input)
* [Model Explainability Monitor](#Model-Explainability-Monitor)
    * [Baselining job](#Baselining-job)
        * [Configurations](#Configurations)
        * [Kick off baselining job](#Kick-off-baselining-job)
    * [Monitoring Schedule](#Monitoring-Schedule)
        * [Wait for the first execution](#Wait-for-the-first-execution)
        * [Wait for the execution to finish](#Wait-for-the-execution-to-finish)
        * [Inspect execution results](#Inspect-execution-results)
* [Cleanup](#Cleanup)

## Introduction

[Amazon SageMaker Model Monitor](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor.html) continuously monitors the quality of Amazon SageMaker machine learning models in production. It enables developers to set alerts for when there are deviations in the model quality. Early and pro-active detection of these deviations enables corrective actions, such as retraining models, auditing upstream systems, or fixing data quality issues without having to monitor models manually or build additional tooling. 

[Amazon SageMaker Clarify Model Explainability Monitor](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-model-monitor-feature-attribution-drift.html) is a model monitor that helps data scientists and ML engineers monitor predictions for feature attribution drift on a regular basis. A drift in the distribution of live data for models in production can result in a corresponding drift in the feature attribution values. As the model is monitored, customers can view exportable reports and graphs detailing feature attributions in SageMaker Studio and configure alerts in Amazon CloudWatch to receive notifications if it is detected that the attribution values drift beyond a certain threshold. 

This notebook demonstrates the process for setting up a [SageMaker Clarify Feature Attribution Drift Monitor](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-model-monitor-feature-attribution-drift.html) for continuous monitoring of feature attribution drift of the data and model used by a regularly running [SageMaker Batch Transform](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform.html) job. The model input and output are in [SageMaker JSON Lines dense format](https://docs.aws.amazon.com/sagemaker/latest/dg/cdf-inference.html#common-in-formats).

In general, you can use the model explainability monitor for batch transform in this way,

1. Schedule a model explainability monitor to monitor a data capture S3 location
1. Regularly run transform jobs with data capture enabled, the jobs save captured data to the data capture S3 URI

The monitor executes processing jobs regularly to do feature attribution analysis, and then generate analysis reports and publish metrics to CloudWatch.

## General Setup

The notebook uses the [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk). The following cell upgrades the SDK and its dependencies. Then you may need to restart the kernel and rerun the notebook to pick up the up-to-date APIs, if the notebook is executed in the SageMaker Studio.

In [1]:
!pip install -U sagemaker
!pip install -U boto3
!pip install -U botocore


[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


### Imports

The following cell imports the APIs to be used by the notebook.

In [2]:
import sagemaker
import pandas as pd
import copy
import datetime
import json
import os
import pprint
import time

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/zicanl/.config/sagemaker/config.yaml


### Handful of configuration

To begin, ensure that these prerequisites have been completed.

* Specify an AWS Region to host the model.
* Specify an IAM role to execute jobs.
* Define the S3 URIs that stores the model file, input data and output data. For demonstration purposes, this notebook uses the same bucket for them. In reality, they could be separated with different security policies.

In [3]:
sagemaker_session = sagemaker.Session()

region = sagemaker_session.boto_region_name
print(f"AWS region: {region}")

role = sagemaker.get_execution_role()
print(f"RoleArn: {role}")

# A different bucket can be used, but make sure the role for this notebook has
# the s3:PutObject permissions. This is the bucket into which the data is captured
bucket = sagemaker_session.default_bucket()
print(f"Demo Bucket: {bucket}")
prefix = sagemaker.utils.unique_name_from_base("sagemaker/DEMO-ClarifyModelMonitor")
print(f"Demo Prefix: {prefix}")
s3_key = f"s3://{bucket}/{prefix}"
print(f"Demo S3 key: {s3_key}")

data_capture_s3_uri = f"{s3_key}/data-capture"
transform_output_s3_uri = f"{s3_key}/transform-output"
baselining_output_s3_uri = f"{s3_key}/baselining-output"
monitor_output_s3_uri = f"{s3_key}/monitor-output"

print(f"The transform job will save the results to: {transform_output_s3_uri}")
print(f"The transform job will save the captured data to: {data_capture_s3_uri}")
print(f"The baselining job will save the analysis results to: {baselining_output_s3_uri}")
print(f"The monitor will save the analysis results to: {monitor_output_s3_uri}")

AWS region: us-west-2
RoleArn: arn:aws:iam::678264136642:role/Admin
Demo Bucket: sagemaker-us-west-2-678264136642
Demo Prefix: sagemaker/DEMO-ClarifyModelMonitor-1705692266-3764
Demo S3 key: s3://sagemaker-us-west-2-678264136642/sagemaker/DEMO-ClarifyModelMonitor-1705692266-3764
The transform job will save the results to: s3://sagemaker-us-west-2-678264136642/sagemaker/DEMO-ClarifyModelMonitor-1705692266-3764/transform-output
The transform job will save the captured data to: s3://sagemaker-us-west-2-678264136642/sagemaker/DEMO-ClarifyModelMonitor-1705692266-3764/data-capture
The baselining job will save the analysis results to: s3://sagemaker-us-west-2-678264136642/sagemaker/DEMO-ClarifyModelMonitor-1705692266-3764/baselining-output
The monitor will save the analysis results to: s3://sagemaker-us-west-2-678264136642/sagemaker/DEMO-ClarifyModelMonitor-1705692266-3764/monitor-output


### Data files

This example includes two dataset files, both in the JSON format.

In [4]:
train_dataset_path = "test_data/validation-dataset.json"
test_dataset_path = "test_data/test-dataset.json"
dataset_type = "application/json"

The train dataset has the features and the ground truth label (pointed to by the key "label"),

In [5]:
!head -n 5 $train_dataset_path

{"instances":[{"features":[41,2,220531,14,15,2,9,0,4,1,0,0,60,38],"label":1},{"features":[33,2,35378,9,13,2,11,5,4,0,0,0,45,38],"label":1},{"features":[36,2,223433,12,14,2,11,0,4,1,7688,0,50,38],"label":1},{"features":[40,2,220589,7,12,4,0,1,4,0,0,0,40,38],"label":0},{"features":[30,2,231413,15,10,2,2,0,4,1,0,0,40,38],"label":1},{"features":[33,4,218164,11,9,2,2,0,4,1,0,0,40,38],"label":0},{"features":[42,2,213464,15,10,2,2,0,4,1,0,0,40,38],"label":0},{"features":[20,2,247794,11,9,4,11,1,4,0,0,0,84,38],"label":0},{"features":[43,2,174575,15,10,0,0,1,4,1,0,0,45,38],"label":0},{"features":[42,4,54202,14,15,2,9,0,4,1,0,0,50,38],"label":1},{"features":[27,2,126060,11,9,4,3,1,4,0,0,0,40,38],"label":0},{"features":[25,2,182866,11,9,4,5,3,4,1,0,0,40,38],"label":0},{"features":[43,2,302041,11,9,4,0,1,2,0,0,0,40,38],"label":0},{"features":[30,2,91145,11,9,4,5,4,4,1,0,0,55,38],"label":0},{"features":[41,2,648223,3,2,3,4,4,4,1,0,0,40,25],"label":0},{"features":[60,2,101096,10,16,4,9,1,4,0,0,0,65,

The test dataset only has features.

In [6]:
!head -n 5 $test_dataset_path

{"instances":[{"features":[28,2,133937,9,13,2,0,0,4,1,15024,0,55,37]},{"features":[43,2,72338,12,14,2,12,0,1,1,0,0,40,37]},{"features":[34,2,162604,11,9,4,2,2,2,1,0,0,40,37]},{"features":[20,2,258509,11,9,4,6,3,2,1,0,0,40,37]},{"features":[27,2,446947,9,13,4,0,4,2,0,0,0,55,37]},{"features":[20,2,95552,11,9,4,11,3,4,1,0,0,40,37]},{"features":[46,2,145636,11,9,2,3,0,4,1,3103,0,50,37]},{"features":[18,2,150675,0,6,4,11,3,4,1,0,0,40,37]},{"features":[22,2,197050,11,9,4,7,3,4,0,0,0,20,37]},{"features":[20,2,246635,15,10,4,11,3,4,0,2597,0,20,37]},{"features":[65,0,200764,11,9,6,0,1,4,0,0,0,40,37]},{"features":[38,2,175665,15,10,2,9,5,4,0,0,0,40,37]},{"features":[34,3,337995,9,13,0,3,4,2,1,15020,0,50,37]},{"features":[42,2,86912,9,13,0,7,1,4,1,0,0,40,37]},{"features":[40,2,100451,15,10,4,2,1,4,1,0,0,40,37]},{"features":[45,2,192360,12,14,2,3,0,4,1,0,1902,50,37]},{"features":[55,2,150507,15,10,2,0,0,4,1,0,0,40,37]},{"features":[36,2,48976,9,13,2,11,5,4,0,0,0,40,37]},{"features":[34,2,111567,15

Here are the headers of the train dataset. "Target" is the header of the ground truth label, and the others are the feature headers. They will be used to beautify the analysis report.

In [7]:
all_headers = [
    "Age",
    "Workclass",
    "fnlwgt",
    "Education",
    "Education-Num",
    "Marital Status",
    "Occupation",
    "Relationship",
    "Ethnic group",
    "Sex",
    "Capital Gain",
    "Capital Loss",
    "Hours per week",
    "Country",
    "Target",
]
label_header = all_headers[-1]

To verify that the execution role for this notebook has the necessary permissions to proceed, put a simple test object into the S3 bucket specified above. If this command fails, update the role to have `s3:PutObject` permission on the bucket and try again.

In [8]:
sagemaker.s3.S3Uploader.upload_string_as_file_body(
    body="hello",
    desired_s3_uri=f"{s3_key}/upload-test-file.txt",
    sagemaker_session=sagemaker_session,
)
print("Success! We are all set to proceed with uploading to S3.")

Success! We are all set to proceed with uploading to S3.


Then upload the data files to S3 so that they can be used by SageMaker jobs.

In [9]:
train_data_s3_uri = sagemaker.s3.S3Uploader.upload(
    local_path=train_dataset_path,
    desired_s3_uri=s3_key,
    sagemaker_session=sagemaker_session,
)
print(f"Train data is uploaded to: {train_data_s3_uri}")
test_data_s3_uri = sagemaker.s3.S3Uploader.upload(
    local_path=test_dataset_path,
    desired_s3_uri=s3_key,
    sagemaker_session=sagemaker_session,
)
print(f"Test data is uploaded to: {test_data_s3_uri}")

Train data is uploaded to: s3://sagemaker-us-west-2-678264136642/sagemaker/DEMO-ClarifyModelMonitor-1705692266-3764/validation-dataset.json
Test data is uploaded to: s3://sagemaker-us-west-2-678264136642/sagemaker/DEMO-ClarifyModelMonitor-1705692266-3764/test-dataset.json


### SageMaker model

This example includes a prebuilt [SageMaker Linear Learner](https://docs.aws.amazon.com/sagemaker/latest/dg/linear-learner.html) model trained by [a SageMaker Clarify offline processing example notebook](https://github.com/aws/amazon-sagemaker-examples/blob/main/sagemaker-clarify/fairness_and_explainability/fairness_and_explainability_jsonlines_format.ipynb). The model supports [SageMaker JSON Lines dense format](https://docs.aws.amazon.com/sagemaker/latest/dg/cdf-inference.html#common-in-formats) (MIME type `"application/jsonlines"`).

* The model input can one or more lines, each line is a JSON object that has a "features" key pointing to a list of feature values concerning demographic characteristics of individuals. For example,

```
{"features":[28,2,133937,9,13,2,0,0,4,1,15024,0,55,37]}
{"features":[43,2,72338,12,14,2,12,0,1,1,0,0,40,37]}
```

* The model output has the predictions of whether a person has a yearly income that is more than $50,000. Each prediction is a JSON object that has a "predicted_label" key pointing to the predicted label, and the "score" key pointing to the confidence score. For example,

```
{"predicted_label":1,"score":0.989977359771728}
{"predicted_label":1,"score":0.504138827323913}
```

In [10]:
model_file = "model/ll-adult-prediction-model.tar.gz"
model_url = sagemaker.s3.S3Uploader.upload(
    local_path=model_file,
    desired_s3_uri=s3_key,
    sagemaker_session=sagemaker_session,
)
print(f"Model file has been uploaded to {model_url}")

model_name = sagemaker.utils.unique_name_from_base("DEMO-xgb-churn-pred-model-monitor")
print(f"SageMaker model name: {model_name}")

image_uri = sagemaker.image_uris.retrieve("linear-learner", region, "1")
print(f"SageMaker Linear Learner image: {image_uri}")

model = sagemaker.model.Model(image_uri=image_uri, model_data=model_url, role=role)
container_def = model.prepare_container_def()
sagemaker_session.create_model(model_name, role, container_def)
print("SageMaker model created")

Model file has been uploaded to s3://sagemaker-us-west-2-678264136642/sagemaker/DEMO-ClarifyModelMonitor-1705692266-3764/ll-adult-prediction-model.tar.gz
SageMaker model name: DEMO-xgb-churn-pred-model-monitor-1705692267-227f
SageMaker Linear Learner image: 174872318107.dkr.ecr.us-west-2.amazonaws.com/linear-learner:1
SageMaker model created


## Batch Transform Job

For continuous monitoring, batch transform jobs should be executed regularly with the latest data. But for demonstration purpose, the following cell only executes the job once before the monitor is scheduled, so that the first monitoring execution has captured data to process. 

See [Transformer](https://sagemaker.readthedocs.io/en/stable/api/inference/transformer.html#sagemaker.transformer.Transformer.transform) for the API reference. The `destination_s3_uri` is used to specify the data capture S3 URI which is a key connection between the job and the monitor.

**NOTE**: The following cell takes about 5 minutes to run.

In [11]:
transfomer = model.transformer(
    instance_count=1,
    instance_type="ml.m5.xlarge",
    accept=dataset_type,  # The transform output data format
    assemble_with=None,  # JSON records are under a single JSON structure
    output_path=transform_output_s3_uri,
)

transfomer.transform(
    data=test_data_s3_uri,
    content_type=dataset_type,  # The transform input format
    split_type=None,  # JSON records are under a single JSON structure
    batch_data_capture_config=sagemaker.inputs.BatchDataCaptureConfig(
        destination_s3_uri=data_capture_s3_uri,
    ),
    wait=True,  # In real world you don't have to wait, but for demo purpose we wait for the output
    logs=False,  # You can change it to True to view job logs inline
)

INFO:sagemaker:Creating transform job with name: linear-learner-2024-01-19-19-24-28-808


....................................................................!


### Captured data

Once the transform job completed, an "input" folders is created under `data_capture_s3_uri`, to includes the captured data files of transform input. Note that, batch transform data capture is unlike endpoint data capture, it does not capture the data for real as it will create tremendous amount of duplications. Instead, it generates [manifest](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_S3DataSource.html#sagemaker-Type-S3DataSource-S3Uri) files which refer to the transform output S3 location.

Now list the captured data files stored in Amazon S3. There should be different files from different time periods organized based on the hour in which the batch transformation occurred. The format of the Amazon S3 path is:

`s3://{data_capture_s3_uri}/input/yyyy/mm/dd/hh/filename.jsonl`

In [12]:
data_capture_output = f"{data_capture_s3_uri}/input"
captured_data_files = sorted(
    sagemaker.s3.S3Downloader.list(
        s3_uri=data_capture_output,
        sagemaker_session=sagemaker_session,
    )
)
print("Found capture data files:")
print("\n ".join(captured_data_files[-5:]))

Found capture data files:
s3://sagemaker-us-west-2-678264136642/sagemaker/DEMO-ClarifyModelMonitor-1705692266-3764/data-capture/input/2024/01/19/19/1b4bda25-59a8-476c-9bb4-65495d56a050.json


In [13]:
captured_data_file = captured_data_files[-1]
captured_data_file_content = sagemaker.s3.S3Downloader.read_file(
    s3_uri=captured_data_files[-1],
    sagemaker_session=sagemaker_session,
)
data_capture_input_dict = json.loads(captured_data_file_content)
print(json.dumps(data_capture_input_dict, indent=4))

[
    {
        "prefix": "s3://sagemaker-us-west-2-678264136642/sagemaker/DEMO-ClarifyModelMonitor-1705692266-3764/test-dataset.json"
    },
    ""
]


In [14]:
def upload_captured_data(offset):
    yyyy_mm_dd_hh = "%Y/%m/%d/%H"
    file_path, file_name = os.path.split(captured_data_file)
    this_hour_str = file_path[len(data_capture_output) + 1 :]  # like "2023/01/18/22"
    this_hour = datetime.datetime.strptime(this_hour_str, yyyy_mm_dd_hh)
    next_hour = this_hour + datetime.timedelta(hours=offset)
    next_hour_str = next_hour.strftime(yyyy_mm_dd_hh)  # like "2023/01/18/23"
    sagemaker.s3.S3Uploader.upload_string_as_file_body(
        body=captured_data_file_content,
        desired_s3_uri=f"{data_capture_output}/{next_hour_str}/{file_name}",
        sagemaker_session=sagemaker_session,
    )


# For demostration purpose, only needed for this example
# copy the captured file to the last hour's folder, just in case the first monitoring execution is started in this hour.
upload_captured_data(-1)
# copy the captured file to the next hour's folder, just in case the first monitoring execution is started after next hour.
upload_captured_data(1)

### Transform input

The captured data file refers to the transform input file. The cell below shows the first few records of the file.

In [15]:
transform_input = data_capture_input_dict[0]["prefix"]
transform_output_content = sagemaker.s3.S3Downloader.read_file(
    s3_uri=transform_input,
    sagemaker_session=sagemaker_session,
).splitlines()
print(*transform_output_content[-5:], sep="\n")

{"instances":[{"features":[28,2,133937,9,13,2,0,0,4,1,15024,0,55,37]},{"features":[43,2,72338,12,14,2,12,0,1,1,0,0,40,37]},{"features":[34,2,162604,11,9,4,2,2,2,1,0,0,40,37]},{"features":[20,2,258509,11,9,4,6,3,2,1,0,0,40,37]},{"features":[27,2,446947,9,13,4,0,4,2,0,0,0,55,37]},{"features":[20,2,95552,11,9,4,11,3,4,1,0,0,40,37]},{"features":[46,2,145636,11,9,2,3,0,4,1,3103,0,50,37]},{"features":[18,2,150675,0,6,4,11,3,4,1,0,0,40,37]},{"features":[22,2,197050,11,9,4,7,3,4,0,0,0,20,37]},{"features":[20,2,246635,15,10,4,11,3,4,0,2597,0,20,37]},{"features":[65,0,200764,11,9,6,0,1,4,0,0,0,40,37]},{"features":[38,2,175665,15,10,2,9,5,4,0,0,0,40,37]},{"features":[34,3,337995,9,13,0,3,4,2,1,15020,0,50,37]},{"features":[42,2,86912,9,13,0,7,1,4,1,0,0,40,37]},{"features":[40,2,100451,15,10,4,2,1,4,1,0,0,40,37]},{"features":[45,2,192360,12,14,2,3,0,4,1,0,1902,50,37]},{"features":[55,2,150507,15,10,2,0,0,4,1,0,0,40,37]},{"features":[36,2,48976,9,13,2,11,5,4,0,0,0,40,37]},{"features":[34,2,111567,15

## Model Explainability Monitor

Similar to the other monitoring types, the standard procedure of creating a [feature attribution drift monitor](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-model-monitor-feature-attribution-drift.html) is first run a baselining job, and then schedule the monitor.

In [16]:
model_explainability_monitor = sagemaker.model_monitor.ModelExplainabilityMonitor(
    role=role,
    sagemaker_session=sagemaker_session,
    max_runtime_in_seconds=3600,
)

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: 1.0.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


### Baselining job

A baselining job runs predictions on training dataset and suggests constraints. The `suggest_baseline()` method of `ModelExplainabilityMonitor` starts a SageMaker Clarify processing job to generate the constraints.

The step is not mandatory, but providing constraints file to the monitor can enable violations file generation.

#### Configurations

Information about the input data need to be provided to the processor.

`DataConfig` stores information about the dataset to be analyzed. For example, the dataset file and its format (like JSON Lines), where to store the analysis results. Some special things to note about this configuration for the JSON Lines dataset,

* The parameter value `"features"` or `"label"` is **NOT** a header string. Instead, it is a `JMESPath` expression ([refer to its specification](https://jmespath.org/specification.html)) that is used to locate the features list or the ground truth label in the dataset (the ground truth label is not needed for the explainability analysis, the parameter is specified so that the job knows it should be excluded from the dataset). In this example notebook they happen to be the same as the keys in the dataset. But for example, if the dataset has records like below, then the `features` parameter should use value `"data.features.values"`, and the `label` parameter should use value `"data.label"`.

  ```
  {"data": {"features": {"values": [25, 2, 226802, 1, 7, 4, 6, 3, 2, 1, 0, 0, 40, 37]}, "label": 0}}
  ```

* SageMaker Clarify processing job will load the JSON Lines dataset into tabular representation for further analysis, and the parameter `headers` is the list of column names. **The label header shall be the last one in the headers list**, and the order of feature headers shall be the same as the order of features in a record.

In [17]:
features_jmespath = "instances[*].features"
ground_truth_label_jmespath = "instances[*].label"
data_config = sagemaker.clarify.DataConfig(
    s3_data_input_path=train_data_s3_uri,
    s3_output_path=baselining_output_s3_uri,
    features=features_jmespath,
    label=ground_truth_label_jmespath,
    headers=all_headers,
    dataset_type=dataset_type,
)

`ModelConfig` is configuration related to model to be used for inferencing. In order to compute SHAP values, the SageMaker Clarify explainer generates synthetic dataset and then get its predictions for the SageMaker model. To accomplish this, the processing job will use the model to create an ephemeral endpoint (also known as "shadow endpoint"). The processing job will delete the shadow endpoint after the computations are completed. One special thing to note about this configuration for the JSON Lines model input and output,

* `content_template` is used by SageMaker Clarify processing job to convert the tabular data to the request payload acceptable to the shadow endpoint. To be more specific, the placeholder `$features` will be replaced by **the features list** from records. The request payload of a record from the testing dataset happens to be similar to the record itself, like `{"features":[28,2,133937,9,13,2,0,0,4,1,15024,0,55,37]}`, because both the dataset and the model input conform to the same format.

In [18]:
model_config = sagemaker.clarify.ModelConfig(
    model_name=model_name,  # The name of the SageMaker model
    instance_type="ml.m5.xlarge",  # The instance type of the shadow endpoint
    instance_count=1,  # The instance count of the shadow endpoint
    content_type=dataset_type,  # The data format of the model input
    accept_type=dataset_type,  # The data format of the model output
    content_template='{"instances":$records}',
    record_template='{"features":$features}',
)

Currently, the SageMaker Clarify explainer offers a scalable and efficient implementation of SHAP, so the explainability config is `SHAPConfig`, including

* `baseline`: A list of records (at least one) to be used as the baseline dataset in the Kernel SHAP algorithm, each record is JSON object that includes a list of features. It can also be a S3 object URI, the S3 file should be in the same format as dataset.
* `num_samples`: Number of samples to be used in the Kernel SHAP algorithm. This number determines the size of the generated synthetic dataset to compute the SHAP values.
* `agg_method`: Aggregation method for global SHAP values. Valid values are
  * "mean_abs" (mean of absolute SHAP values for all instances),
  * "median" (median of SHAP values for all instances) and
  * "mean_sq" (mean of squared SHAP values for all instances).
* `use_logit`: Indicator of whether the logit function is to be applied to the model predictions. Default is False. If "use_logit" is true then the SHAP values will have log-odds units.
* `save_local_shap_values`: Indicator of whether to save the local SHAP values in the output location. Default is True.

In [19]:
# Here use the mean value of train dataset as SHAP baseline
dataset = []
with open(train_dataset_path) as f:
    instances = json.load(f)["instances"]
    for instance in instances:
        dataset.append(instance["features"])
mean_values = pd.DataFrame(dataset).mean().round().astype(int).to_list()
mean_record = {"features": mean_values}
shap_baseline = {"instances": [mean_record]}
print(f"SHAP baseline: {shap_baseline}")

shap_config = sagemaker.clarify.SHAPConfig(
    baseline=shap_baseline,
    num_samples=100,
    agg_method="mean_abs",
    save_local_shap_values=False,
)

SHAP baseline: {'instances': [{'features': [39, 2, 184870, 10, 10, 3, 6, 1, 4, 1, 1597, 61, 41, 37]}]}


#### Kick off baselining job

Call the `suggest_baseline()` method to start the baselining job. The model output has a key "score" pointing to a confidence score value between `0` and `1`. So, the `model_scores` parameter is set to the `JMESPath` expression"score" which can locate the score in the model output.

In [20]:
confidence_score_jmespath = "predictions[*].score"
model_explainability_monitor.suggest_baseline(
    explainability_config=shap_config,
    data_config=data_config,
    model_config=model_config,
    model_scores=confidence_score_jmespath,  # The JMESPath to locate the confidence score in model output
)

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: 1.0.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker.clarify:Analysis Config: {'dataset_type': 'application/json', 'features': 'instances[*].features', 'headers': ['Age', 'Workclass', 'fnlwgt', 'Education', 'Education-Num', 'Marital Status', 'Occupation', 'Relationship', 'Ethnic group', 'Sex', 'Capital Gain', 'Capital Loss', 'Hours per week', 'Country', 'Target'], 'label': 'instances[*].label', 'predictor': {'model_name': 'DEMO-xgb-churn-pred-model-monitor-1705692267-227f', 'instance_type': 'ml.m5.xlarge', 'initial_instance_count': 1, 'accept_type': 'application/json', 'content_type': 'application/json', 'content_template': '{"instances":$records}', 'record_template': '{"features":$features}', 'label': 'predictions[*].score'}, 'methods': {'report': {'name': 'report', 'title': 'Analysis Report'}, 'shap': {'use_logit': False, 'save_local_shap_values': False, 'baselin

**NOTE**: The following cell waits until the baselining job is completed (in about 10 minutes). It then inspects the suggested constraints. This step can be skipped, because the monitor to be scheduled will automatically pick up baselining job name and wait for it before monitoring execution.

In [21]:
model_explainability_monitor.latest_baselining_job.wait(logs=False)
print()
model_explainability_constraints = model_explainability_monitor.suggested_constraints()
print(f"Suggested constraints: {model_explainability_constraints.file_s3_uri}")
print(
    sagemaker.s3.S3Downloader.read_file(
        s3_uri=model_explainability_constraints.file_s3_uri,
        sagemaker_session=sagemaker_session,
    )
)

.......................................................................................................!
Suggested constraints: s3://sagemaker-us-west-2-678264136642/sagemaker/DEMO-ClarifyModelMonitor-1705692266-3764/baselining-output/analysis.json
{
    "version": "1.0",
    "explanations": {
        "kernel_shap": {
            "0.7956783771514893": {
                "global_shap_values": {
                    "Age": 0.05967323398925334,
                    "Workclass": 0.009303977202541277,
                    "fnlwgt": 0.0011532925777696535,
                    "Education": 0.014668402906540028,
                    "Education-Num": 0.09897500295961109,
                    "Marital Status": 0.05465943541380248,
                    "Occupation": 0.002534111174299059,
                    "Relationship": 0.018197139997990445,
                    "Ethnic group": 0.005443095081746528,
                    "Sex": 0.03218866814815311,
                    "Capital Gain": 0.09933718978948747,

### Monitoring Schedule

With above constraints collected, now call `create_monitoring_schedule()` method to schedule an hourly model explainability monitor.

If a baselining job has been submitted, then the monitor object will automatically pick up the analysis configuration from the baselining job. But if the baselining step is skipped, or if the capture dataset has different nature than the training dataset, then analysis configuration has to be provided.

`ModelConfig` is required by `ExplainabilityAnalysisConfig` for the same reason as it is required by the baselining job. Note that only features are required for computing feature attribution, so ground truth label should be excluded.

Highlights,

* `data_capture_s3_uri` is the location of data captured by the batch transform job
* `features_attribute` is the `JMESPath` expression to locate the features in model input, similar to the `features` parameter of `DataConfig`.
* `inference_attribute` stores the `JMESPath` expression to locate the confidence score in model output, similar to the `model_scores` parameter of the `suggest_baseline()` method.

In [22]:
schedule_expression = sagemaker.model_monitor.CronExpressionGenerator.hourly()

In [23]:
# Remove label because only features are required for the analysis
headers_without_label_header = copy.deepcopy(all_headers)
headers_without_label_header.remove(label_header)
model_explainability_analysis_config = sagemaker.model_monitor.ExplainabilityAnalysisConfig(
    explainability_config=shap_config,
    model_config=model_config,
    headers=headers_without_label_header,
)
model_explainability_monitor.create_monitoring_schedule(
    analysis_config=model_explainability_analysis_config,
    batch_transform_input=sagemaker.model_monitor.BatchTransformInput(
        data_captured_destination_s3_uri=data_capture_s3_uri,
        destination="/opt/ml/processing/transform",
        dataset_format=sagemaker.model_monitor.MonitoringDatasetFormat.json(lines=False),
        features_attribute=features_jmespath,
        inference_attribute=confidence_score_jmespath,
    ),
    output_s3_uri=monitor_output_s3_uri,
    schedule_cron_expression=schedule_expression,
)
print(
    f"Model explainability monitoring schedule: {model_explainability_monitor.monitoring_schedule_name}"
)

INFO:sagemaker.model_monitor.clarify_model_monitoring:Uploading analysis config to {s3_uri}.
INFO:sagemaker.model_monitor.model_monitoring:Creating Monitoring Schedule with name: monitoring-schedule-2024-01-19-19-39-01-339


Model explainability monitoring schedule: monitoring-schedule-2024-01-19-19-39-01-339


#### Wait for the first execution

The schedule starts jobs at the previously specified intervals. Code below waits until time crosses the hour boundary (in UTC) to see executions kick off.

Note: Even for an hourly schedule, Amazon SageMaker has a buffer period of 20 minutes to schedule executions. The execution might start in anywhere from zero to ~20 minutes from the hour boundary. This is expected and done for load balancing in the backend.

In [24]:
def wait_for_execution_to_start(model_monitor):
    print(
        "An hourly schedule was created above and it will kick off executions ON the hour (plus 0 - 20 min buffer)."
    )

    print("Waiting for the first execution to happen", end="")
    schedule_desc = model_monitor.describe_schedule()
    while "LastMonitoringExecutionSummary" not in schedule_desc:
        schedule_desc = model_monitor.describe_schedule()
        print(".", end="", flush=True)
        time.sleep(60)
    print()
    print("Done! Execution has been created")

    print("Now waiting for execution to start", end="")
    while schedule_desc["LastMonitoringExecutionSummary"]["MonitoringExecutionStatus"] in "Pending":
        schedule_desc = model_monitor.describe_schedule()
        print(".", end="", flush=True)
        time.sleep(10)

    print()
    print("Done! Execution has started")

**NOTE**: The following cell waits until the first monitoring execution is started. As explained above, the wait could take more than 60 minutes.

In [25]:
wait_for_execution_to_start(model_explainability_monitor)

An hourly schedule was created above and it will kick off executions ON the hour (plus 0 - 20 min buffer).
Waiting for the first execution to happen...............................
Done! Execution has been created
Now waiting for execution to start..........
Done! Execution has started


In real world, a monitoring schedule is supposed to be active all the time. But in this example, it can be stopped to avoid incurring extra charges. A stopped schedule will not trigger further executions, but the ongoing execution will continue. And if needed, the schedule can be restarted by `start_monitoring_schedule()`.

In [26]:
model_explainability_monitor.stop_monitoring_schedule()

INFO:sagemaker:Stopping Monitoring Schedule with name: monitoring-schedule-2024-01-19-19-39-01-339


#### Wait for the execution to finish

In the previous cell, the first execution has started. This section waits for the execution to finish so that its analysis results are available. Here are the possible terminal states and what each of them mean:

* `Completed` - This means the monitoring execution completed, and no issues were found in the violations report.
* `CompletedWithViolations` - This means the execution completed, but constraint violations were detected.
* `Failed` - The monitoring execution failed, maybe due to client error (perhaps incorrect role permissions) or infrastructure issues. Further examination of `FailureReason` and `ExitMessage` is necessary to identify what exactly happened.
* `Stopped` - job exceeded max runtime or was manually stopped.

In [27]:
# Waits for the schedule to have last execution in a terminal status.
def wait_for_execution_to_finish(model_monitor):
    schedule_desc = model_monitor.describe_schedule()
    execution_summary = schedule_desc.get("LastMonitoringExecutionSummary")
    if execution_summary is not None:
        print("Waiting for execution to finish", end="")
        while execution_summary["MonitoringExecutionStatus"] not in [
            "Completed",
            "CompletedWithViolations",
            "Failed",
            "Stopped",
        ]:
            print(".", end="", flush=True)
            time.sleep(60)
            schedule_desc = model_monitor.describe_schedule()
            execution_summary = schedule_desc["LastMonitoringExecutionSummary"]
        print()
        print(f"Done! Execution Status: {execution_summary['MonitoringExecutionStatus']}")
    else:
        print("Last execution not found")

**NOTE**: The following cell takes about 10 minutes.

In [28]:
wait_for_execution_to_finish(model_explainability_monitor)

Waiting for execution to finish..........
Done! Execution Status: Completed


#### Inspect execution results

List the generated reports,

* analysis.json includes the global SHAP values.
* report.* files are static report files to visualize the SHAP values.

In [29]:
schedule_desc = model_explainability_monitor.describe_schedule()
execution_summary = schedule_desc.get("LastMonitoringExecutionSummary")
if execution_summary and execution_summary["MonitoringExecutionStatus"] in [
    "Completed",
    "CompletedWithViolations",
]:
    last_model_explainability_monitor_execution = model_explainability_monitor.list_executions()[-1]
    last_model_explainability_monitor_execution_report_uri = (
        last_model_explainability_monitor_execution.output.destination
    )
    print(f"Report URI: {last_model_explainability_monitor_execution_report_uri}")
    last_model_explainability_monitor_execution_report_files = sorted(
        sagemaker.s3.S3Downloader.list(
            s3_uri=last_model_explainability_monitor_execution_report_uri,
            sagemaker_session=sagemaker_session,
        )
    )
    print("Found Report Files:")
    print("\n ".join(last_model_explainability_monitor_execution_report_files))
else:
    last_model_explainability_monitor_execution = None
    print(
        "====STOP==== \n No completed executions to inspect further. Please wait till an execution completes or investigate previously reported failures."
    )
    print(schedule_desc)

Report URI: s3://sagemaker-us-west-2-678264136642/sagemaker/DEMO-ClarifyModelMonitor-1705692266-3764/monitor-output/monitoring-schedule-2024-01-19-19-39-01-339/2024/01/19/20
Found Report Files:
s3://sagemaker-us-west-2-678264136642/sagemaker/DEMO-ClarifyModelMonitor-1705692266-3764/monitor-output/monitoring-schedule-2024-01-19-19-39-01-339/2024/01/19/20/analysis.json
 s3://sagemaker-us-west-2-678264136642/sagemaker/DEMO-ClarifyModelMonitor-1705692266-3764/monitor-output/monitoring-schedule-2024-01-19-19-39-01-339/2024/01/19/20/report.html
 s3://sagemaker-us-west-2-678264136642/sagemaker/DEMO-ClarifyModelMonitor-1705692266-3764/monitor-output/monitoring-schedule-2024-01-19-19-39-01-339/2024/01/19/20/report.ipynb
 s3://sagemaker-us-west-2-678264136642/sagemaker/DEMO-ClarifyModelMonitor-1705692266-3764/monitor-output/monitoring-schedule-2024-01-19-19-39-01-339/2024/01/19/20/report.pdf


If there are any violations compared to the baseline, they are listed here. See [Feature Attribution Drift Violations](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-model-monitor-model-attribution-drift-violations.html) for the schema of the file, and how violations are detected.

In [30]:
violations = model_explainability_monitor.latest_monitoring_constraint_violations()
if violations is not None:
    pprint.PrettyPrinter(indent=4).pprint(violations.body_dict)


Could not retrieve constraints file at location 's3://sagemaker-us-west-2-678264136642/sagemaker/DEMO-ClarifyModelMonitor-1705692266-3764/monitor-output/monitoring-schedule-2024-01-19-19-39-01-339/2024/01/19/20/constraint_violations.json'. To manually retrieve ConstraintViolations object from a given uri, use 'my_model_monitor.constraints(my_s3_uri)' or 'ConstraintViolations.from_s3_uri(my_s3_uri)'


By default, the analysis results are also published to CloudWatch, see [CloudWatch Metrics for Feature Attribution Drift Analysis](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-feature-attribute-drift-cw.html).

## Cleanup

If there is no plan to collect more data for feature attribution drift monitoring, then the monitor should be stopped (and deleted) to avoid incurring additional charges. Note that deleting the monitor does not delete the data in S3.

In [31]:
model_explainability_monitor.stop_monitoring_schedule()
wait_for_execution_to_finish(model_explainability_monitor)
model_explainability_monitor.delete_monitoring_schedule()

INFO:sagemaker:Stopping Monitoring Schedule with name: monitoring-schedule-2024-01-19-19-39-01-339
INFO:sagemaker:Deleting Monitoring Schedule with name: monitoring-schedule-2024-01-19-19-39-01-339


Waiting for execution to finish
Done! Execution Status: Completed


INFO:sagemaker.model_monitor.clarify_model_monitoring:Deleting Model Explainability Job Definition with name: model-explainability-job-definition-2024-01-19-19-39-01-339


In [32]:
sagemaker_session.delete_model(model_name)

INFO:sagemaker:Deleting model with name: DEMO-xgb-churn-pred-model-monitor-1705692267-227f


## Notebook CI Test Results

This notebook was tested in multiple regions. The test results are as follows, except for us-west-2 which is shown at the top of the notebook.


![This us-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-1/sagemaker_model_monitor|fairness_and_explainability_json|SageMaker-Monitoring-Feature-Attribution-Drift-for-Batch-Transform.ipynb)

![This us-east-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-2/sagemaker_model_monitor|fairness_and_explainability_json|SageMaker-Monitoring-Feature-Attribution-Drift-for-Batch-Transform.ipynb)

![This us-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-1/sagemaker_model_monitor|fairness_and_explainability_json|SageMaker-Monitoring-Feature-Attribution-Drift-for-Batch-Transform.ipynb)

![This ca-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ca-central-1/sagemaker_model_monitor|fairness_and_explainability_json|SageMaker-Monitoring-Feature-Attribution-Drift-for-Batch-Transform.ipynb)

![This sa-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/sa-east-1/sagemaker_model_monitor|fairness_and_explainability_json|SageMaker-Monitoring-Feature-Attribution-Drift-for-Batch-Transform.ipynb)

![This eu-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-1/sagemaker_model_monitor|fairness_and_explainability_json|SageMaker-Monitoring-Feature-Attribution-Drift-for-Batch-Transform.ipynb)

![This eu-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-2/sagemaker_model_monitor|fairness_and_explainability_json|SageMaker-Monitoring-Feature-Attribution-Drift-for-Batch-Transform.ipynb)

![This eu-west-3 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-3/sagemaker_model_monitor|fairness_and_explainability_json|SageMaker-Monitoring-Feature-Attribution-Drift-for-Batch-Transform.ipynb)

![This eu-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-central-1/sagemaker_model_monitor|fairness_and_explainability_json|SageMaker-Monitoring-Feature-Attribution-Drift-for-Batch-Transform.ipynb)

![This eu-north-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-north-1/sagemaker_model_monitor|fairness_and_explainability_json|SageMaker-Monitoring-Feature-Attribution-Drift-for-Batch-Transform.ipynb)

![This ap-southeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-1/sagemaker_model_monitor|fairness_and_explainability_json|SageMaker-Monitoring-Feature-Attribution-Drift-for-Batch-Transform.ipynb)

![This ap-southeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-2/sagemaker_model_monitor|fairness_and_explainability_json|SageMaker-Monitoring-Feature-Attribution-Drift-for-Batch-Transform.ipynb)

![This ap-northeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-1/sagemaker_model_monitor|fairness_and_explainability_json|SageMaker-Monitoring-Feature-Attribution-Drift-for-Batch-Transform.ipynb)

![This ap-northeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-2/sagemaker_model_monitor|fairness_and_explainability_json|SageMaker-Monitoring-Feature-Attribution-Drift-for-Batch-Transform.ipynb)

![This ap-south-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-south-1/sagemaker_model_monitor|fairness_and_explainability_json|SageMaker-Monitoring-Feature-Attribution-Drift-for-Batch-Transform.ipynb)

